In [ ]:
import matplotlib.pyplot as plt
import cv2
import importlib

import scipy

from pathlib import Path

import numpy as np

import h5py
import math


import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import holoviews as hv
hv.extension('bokeh')

import platform

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../../..")


from pathlib import Path

import cv2

import json

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook
import iqplot
import bokeh.io
bokeh.io.output_notebook()

import Utilities.Ballpushing_utils
from Utilities.Utils import *
from Utilities.Processing import *
from Utilities.Ballpushing_utils import *

# Get a list of the directories containing the tracking data

In [ ]:
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)

Make a list of the folders I want to use. For instance, I want to use the folders that have the "tnt" in the name as I will explore velocities for different crossings with UAS-TNT. I'm also only getting flies tested in the afternoon.

## Vision + Starvation state effect

In [ ]:
Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "feedingstate" in minfolder and "tracked" in minfolder and "pm" in minfolder and 'flipped' not in minfolder and 'flip' not in minfolder:
        Folders.append(folder)

Folders

## Importing the tracking data and generating the dataset

In this part, we import the Metatadata .json file and the tracking data .h5 file. Then we compute smoothed fly y positions and generate time column.

In [ ]:
Dataset = generate_dataset(Folders)

In [ ]:
GroupedData = Dataset.groupby(["time", "Light", "FeedingState"]).mean(numeric_only=True).reset_index()

GroupedData.head()

In [ ]:

#GroupedData.loc[GroupedData['time'] < 0.04]

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Define a function to return alpha value based on Light condition
def alpha_val(light_condition):
    return 0.5 if light_condition == 'off' else 1.0

# Apply the function to the dataset to create a new column 'alpha'
GroupedData['alpha'] = GroupedData['Light'].apply(alpha_val)

color_dict = {"fed": "blue", "starved": "orange", "starved_noWater": "green"}

# Create an empty overlay
overlay = hv.NdOverlay({})

# Create separate overlays for each FeedingState
for feeding_state, color in color_dict.items():
    # Filter data for the current FeedingState
    data = GroupedData[GroupedData['FeedingState'] == feeding_state]
    
    # Create curves and overlay them
    curves = hv.Curve(data, kdims=['time'], vdims=['yball_relative', 'Light', 'alpha'])
    curves = curves.groupby('Light').overlay().opts(opts.Curve(color=color, alpha='alpha'))
    
    # Add to the overall overlay
    overlay[feeding_state] = curves

overlay = overlay.opts( 
        height=750,
        width=1000,
        #alpha=1,
        #line_width=2,
        xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=2,
        title="",)
# Display the plot
overlay


In [ ]:
import holoviews as hv
from holoviews import opts

# Define a function to map FeedingState to color
def feedingstate_to_color(feedingstate):
    color_dict = {"fed": "blue", "starved": "orange", "starved_noWater": "green"}
    return color_dict.get(feedingstate, 0)

# Define a function to map Light condition to alpha
def light_to_alpha(light):
    return 0.5 if light == 'off' else 1.0  # replace with your actual logic of mapping light to alpha

# Apply the mappings to FeedingState and Light condition
GroupedData['FeedingStateColor'] = GroupedData['FeedingState'].apply(feedingstate_to_color)
#print(GroupedData['FeedingStateColor'])
GroupedData['LightAlpha'] = GroupedData['Light'].apply(light_to_alpha)

# Define a function to map FeedingState to new labels
def feedingstate_to_label(feedingstate):
    label_dict = {"fed": "fed", "starved": "starved with water", "starved_noWater": "starved without water"}
    return label_dict.get(feedingstate, feedingstate)

# Apply the mapping to FeedingState
GroupedData['FeedingStateLabel'] = GroupedData['FeedingState'].apply(feedingstate_to_label)

# Calculate the sample size
sample_size = Dataset.groupby(['Light','FeedingState'])['Fly'].nunique()

# Define a function to map FeedingState, Light, and sample size to new labels
def feedingstate_light_to_label(row):
    label = row['FeedingStateLabel']
    light = row['Light']
    size = sample_size.loc[light, row['FeedingState']]
    return f"{label}, {light} (n={size})"

# Apply the mapping to FeedingState, Light, and sample size
GroupedData['Feeding state, Light'] = GroupedData.apply(feedingstate_light_to_label, axis=1)

# Create the curves and apply the options
curves = hv.Curve(GroupedData, kdims=['time'], vdims=['yball_relative','FeedingStateLabel', 'Light', 'FeedingStateColor', 'LightAlpha', 'alpha', 'Feeding state, Light']).groupby(['Feeding state, Light']).overlay()
curves.opts(opts.Curve(color='FeedingStateColor', alpha='LightAlpha', 
        height=750,
        width=1000,
        line_width=1,
        xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=1.5,
        title="",
        ))
curves.opts(legend_position='bottom_right')

curves




In [ ]:
# Save the plot as a png
hv.save(
    curves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/feeding_and_light.png",
    fmt="png",
)

In [ ]:
# Save the plot as a html
hv.save(
    curves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/feeding_and_light.html",
    fmt="html",
)

In [ ]:
import holoviews as hv
from holoviews import opts

# Define a function to map FeedingState to color
def feedingstate_to_color(feedingstate):
    color_dict = {"fed": "blue", "starved": "orange", "starved_noWater": "green"}
    return color_dict.get(feedingstate, 0)

# Define a function to map Light condition to alpha
def light_to_alpha(light):
    return 0.5 if light == 'off' else 1.0

# Apply the mappings to FeedingState and Light condition
GroupedData['FeedingStateColor'] = GroupedData['FeedingState'].apply(feedingstate_to_color)
GroupedData['LightAlpha'] = GroupedData['Light'].apply(light_to_alpha)

# Define a function to map FeedingState to new labels
def feedingstate_to_label(feedingstate):
    label_dict = {"fed": "fed", "starved": "starved with water", "starved_noWater": "starved without water"}
    return label_dict.get(feedingstate, feedingstate)

# Apply the mapping to FeedingState
GroupedData['FeedingStateLabel'] = GroupedData['FeedingState'].apply(feedingstate_to_label)

# Calculate the sample size
sample_size = Dataset.groupby(['Light','FeedingState'])['Fly'].nunique()

# Define a function to map FeedingState, Light, and sample size to new labels
def feedingstate_light_to_label(row):
    label = row['FeedingStateLabel']
    light = row['Light']
    size = sample_size.loc[light, row['FeedingState']]
    return f"{label}, {light} (n={size})"

# Apply the mapping to FeedingState, Light, and sample size
GroupedData['Feeding state, Light'] = GroupedData.apply(feedingstate_light_to_label, axis=1)

# Create separate overlays for each FeedingState and Light condition
overlay_feedingstate = hv.NdOverlay({})
overlay_light = hv.NdOverlay({})

for feeding_state in GroupedData['FeedingState'].unique():
    # Filter data for the current FeedingState
    data_feedingstate = GroupedData[GroupedData['FeedingState'] == feeding_state]
    
    # Create curves and overlay them for FeedingState
    curves_feedingstate = hv.Curve(data_feedingstate, kdims=['time'], vdims=['yball_relative', 'Feeding state, Light']).groupby('Feeding state, Light').overlay()
    
    # Add to the FeedingState overlay
    overlay_feedingstate[feeding_state] = curves_feedingstate.opts(opts.Curve(color='FeedingStateColor', alpha='LightAlpha'))

for light in GroupedData['Light'].unique():
    # Filter data for the current Light condition
    data_light = GroupedData[GroupedData['Light'] == light]
    
    # Create curves and overlay them for Light condition
    curves_light = hv.Curve(data_light, kdims=['time'], vdims=['yball_relative', 'Feeding state, Light']).groupby('Feeding state, Light').overlay()
    
    # Add to the Light condition overlay
    overlay_light[light] = curves_light.opts(opts.Curve(color='FeedingStateColor', alpha='LightAlpha'))

# Set options for the plots
opts_dict = dict(height=750, width=1000, line_width=1, xlabel="Time(s)", ylabel="Average relative distance pushed (px)", show_grid=True, fontscale=1.5, title="", legend_position='bottom_right')

# Apply options to the overlays
overlay_feedingstate = overlay_feedingstate.opts(**opts_dict)
overlay_light = overlay_light.opts(**opts_dict)

# Display the plots side by side
hv.Layout(overlay_feedingstate + overlay_light).cols(1)


In [ ]:
# Attempt to save the datasets 

DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Datasets")

GroupedData.to_feather(DataPath / "230928_GroupedDataFeedingStateLightMean.feather")

Dataset.to_feather(DataPath / "230928_DatasetFeedingStateLight.feather")

## Genotype data

In [ ]:
%reset -f

In [ ]:
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)

In [ ]:
Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "tnt" in minfolder and "tracked" in minfolder and "pm" in minfolder:
        Folders.append(folder)

Folders

## Importing the tracking data and generating the dataset

In this part, we import the Metatadata .json file and the tracking data .h5 file. Then we compute smoothed fly y positions and generate time column.

In [ ]:
Dataset = generate_dataset(Folders, Events = 'interactions')

In [ ]:
Dataset.head()

In [ ]:
GroupOps = Dataset.groupby(
    [
        "time",
        "Genotype",
    ]
)

In [ ]:
GroupData = GroupOps.mean(numeric_only=True).reset_index()

The part below is too heavy to be executed in a notebook. I just ran it in a separate script.

In [ ]:
#Confints = GroupOps["yball_relative"].apply(lambda x: draw_bs_ci(x, n_reps=300))

# DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Datasets")

# #GroupOps.to_feather(DataPath / "230928_GroupedDataTNT.feather")

#Dataset.to_feather(DataPath / "231010_DatasetTNT.feather")

It should now be possible to load the confints from the script output

In [ ]:
DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Datasets")

Confints = pd.read_feather(DataPath / "231010_Confints.feather")

In [ ]:
Confints.head()

In [ ]:
Confints[["ci_lower", "ci_upper"]] = Confints["yball_relative"].tolist()

Confints.head()

In [ ]:
Confints_process = Confints.reset_index()

In [ ]:
# Define GroupData["ci_minus"] and GroupData["ci_plus"] as the columns containing the values of Confints
GroupData["ci_lower"] = Confints_process["ci_lower"]
GroupData["ci_upper"] = Confints_process["ci_upper"]

# Define GroupData["ci_minus"] and GroupData["ci_plus"] as the columns containing the values of Confints

In [ ]:
GroupData.head()

In [ ]:
from bokeh.models import ColumnDataSource, Line, VArea
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="yball over time", x_axis_label='time', y_axis_label='yball_relative')


In [ ]:

# Get the list of genotypes
genotypes = GroupData['Genotype'].unique()

#genotypes = ['PR', 'TNTxTH', 'TNTxE-PG']


In [ ]:

# For each genotype, create a line plot with confidence intervals
for i, genotype in enumerate(genotypes):
    df_genotype = GroupData[GroupData['Genotype'] == genotype]
    source = ColumnDataSource(df_genotype)
    
    # Draw the line for yball_relative
    p.line('time', 'yball_relative', source=source, line_width=2, color=Spectral11[i], legend_label=genotype)
    
    # Draw the upper and lower bounds for the confidence interval
    p.varea(x='time', y1='ci_lower', y2='ci_upper', source=source, fill_color=Spectral11[i], fill_alpha=0.4)

# Show the results
show(p)


In [ ]:
import pandas as pd
import hvplot.pandas  # noqa

# First, we melt the DataFrame to have 'Genotype' as columns for 'yball_relative'
df_melted = GroupData.melt(id_vars=['time'], value_vars=['yball_relative'], col_level=0, var_name='Genotype')

# Then, we create a HoloViews Dataset from the melted DataFrame
ds = hv.Dataset(df_melted)

# Now, we can use the hvPlot interface to plot it
plot = ds.to(hv.Curve, 'time', 'value')

# We also want to display the confidence intervals. For that, we need to melt the 'ci_lower' and 'ci_upper' columns too
df_confidence = GroupData.melt(id_vars=['time'], value_vars=['ci_lower', 'ci_upper'], col_level=0, var_name='Genotype')

# And create a HoloViews Dataset from the melted DataFrame for confidence intervals
ds_confidence = hv.Dataset(df_confidence)

# Create a HoloViews Area between the lower and upper confidence bounds
confidence = ds_confidence.to(hv.Area, 'time', ['ci_lower', 'ci_upper'])

# Finally, we overlay the two plots
final_plot = (plot * confidence).opts(hv.opts.Area(fill_alpha=0.2))

final_plot


In [ ]:
visible_genotypes = ['TNTxE-PG', 'TNTxTH', 'TNTxLAL1'] 

from bokeh.palettes import Category10

# Get the unique genotypes
genotypes = GroupData.Genotype.unique()

# Create a dictionary that maps each genotype to a color
colormap = {genotype: Category10[len(genotypes)][i] for i, genotype in enumerate(genotypes)}

GenoCurves = hv.Overlay([hv.Curve(GroupData.query("Genotype == @genotype"), kdims=["time"], vdims=['yball_relative'], label = genotype).opts(color=colormap[genotype], muted_line_alpha=0, muted = False if genotype in visible_genotypes else True) for genotype in genotypes])

Areas = hv.Overlay([hv.Area(GroupData.query("Genotype == @genotype"), kdims=['time'], vdims=['ci_lower','ci_upper','Genotype'], label = genotype).opts(fill_color=colormap[genotype], fill_alpha= 0.2 if genotype in visible_genotypes else 0, line_alpha=0, muted_fill_alpha=0, muted_line_alpha=0,  muted = False if genotype in visible_genotypes else True) for genotype in genotypes])


hint_text = hv.Text(3500, 100, 'How to navigate: click on genotype labels in the legend to display/mute them.\n Refreshing can be slow, do not click multiple times.\n Downloading the html file locally makes it easier to navigate.', halign='left')

AvgCurves = (GenoCurves 
             * Areas 
             #* hint_text
             ).opts(width=1280, height=720, show_legend=True, xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None
        )
hv.save(
    AvgCurves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/AverageCurvesTNTHV_notext.png",
    #fmt="html",
)

In [ ]:
visible_genotypes = ['TNTxE-PG', 'TNTxTH', 'TNTxLAL1'] 

Sub = GroupData.query("Genotype == @visible_genotypes")

from bokeh.palettes import Category10

# Get the unique genotypes
genotypes = Sub.Genotype.unique()

# Create a dictionary that maps each genotype to a color
colormap = {genotype: Category10[len(genotypes)][i] for i, genotype in enumerate(genotypes)}

GenoCurves = hv.Overlay([hv.Curve(Sub.query("Genotype == @genotype"), kdims=["time"], vdims=['yball_relative'], label = genotype).opts(color=colormap[genotype], muted_line_alpha=0, muted = False if genotype in visible_genotypes else True) for genotype in genotypes])

Areas = hv.Overlay([hv.Area(Sub.query("Genotype == @genotype"), kdims=['time'], vdims=['ci_lower','ci_upper','Genotype'], label = genotype).opts(fill_color=colormap[genotype], fill_alpha= 0.2 if genotype in visible_genotypes else 0, line_alpha=0, muted_fill_alpha=0, muted_line_alpha=0,  muted = False if genotype in visible_genotypes else True) for genotype in genotypes])


hint_text = hv.Text(3500, 100, 'How to navigate: click on genotype labels in the legend to display/mute them.\n Refreshing can be slow, do not click multiple times.\n Downloading the html file locally makes it easier to navigate.', halign='left')

AvgCurves = (GenoCurves 
             * Areas 
             #* hint_text
             ).opts(width=1280, height=720, show_legend=True, xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None
        )
hv.save(
    AvgCurves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/AverageCurvesTNTHV_notext.png",
    #fmt="html",
)

In [ ]:
GenoCurves = hv.Curve(GroupData, kdims=["time"], vdims=['yball_relative', 'Genotype']).groupby('Genotype').opts(muted_line_alpha=0, muted = True).overlay()

Areas = hv.Area(GroupData, kdims=['time'], vdims=['ci_lower','ci_upper','Genotype']).groupby('Genotype').opts(fill_alpha= 0.2, line_alpha=0, muted_fill_alpha=0, muted_line_alpha=0, muted = True).overlay()

hint_text = hv.Text(100, 5000, 'How to navigate: click on genotype labels in the legend to display/mute them.\n Refreshing can be slow, do not click multiple times.\n Downloading the plot locally makes it easier to navigate.')

AvgCurves = (GenoCurves * Areas * hint_text).opts(width=1280, height=720, show_legend=True, xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None)
hv.save(
    AvgCurves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/AverageCurvesTNTHV_muted.html",
    #fmt="html",
)

In [ ]:
import panel as pn
import holoviews as hv

# Create a CheckboxGroup widget with all genotypes
checkboxes = pn.widgets.CheckboxGroup(name='Genotypes', options=list(GroupData.Genotype.unique()), value=['TNTxE-PG', 'TNTxTH', 'TNTxLAL1'])

@pn.depends(checkboxes.param.value)
def plot_genotypes(genotypes):
    GenoCurves = hv.Overlay([hv.Curve(GroupData.query("Genotype == @genotype"), kdims=["time"], vdims=['yball_relative'], label = genotype) for genotype in genotypes])
    Areas = hv.Overlay([hv.Area(GroupData.query("Genotype == @genotype"), kdims=['time'], vdims=['ci_lower','ci_upper','Genotype'], label = genotype) for genotype in genotypes])
    hint_text = hv.Text(100, 5000, 'How to navigate: click on genotype labels in the legend to display/mute them.\n Refreshing can be slow, do not click multiple times.\n Downloading the plot locally makes it easier to navigate.')
    AvgCurves = (GenoCurves * Areas * hint_text).opts(width=1280, height=720, show_legend=True, xlabel="Time(s)",
            ylabel="Average relative distance pushed (px)",
            show_grid=True,
            fontscale=1.5,
            title="",)
    return AvgCurves

# Create a Panel layout with the checkboxes and the plot
layout = pn.Row(checkboxes, plot_genotypes)

# Save the layout as an HTML file
layout.save("/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/AverageCurvesTNTHV_checkbox.html")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import holoviews.operation.datashader as hd

# Create a colormap
cmap = plt.cm.viridis

# Get the colormap colors
my_cmap = cmap(np.arange(cmap.N))

# Set alpha
my_cmap[:,-1] = np.linspace(0, 1, cmap.N)

# Create new colormap
my_cmap = plt.ListedColormap(my_cmap)

visible_genotypes = ['TNTxE-PG', 'TNTxTH', 'TNTxLAL1'] 

GenoCurves = hv.Overlay([hd.datashade(hv.Curve(GroupData.query("Genotype == @genotype"), kdims=["time"], vdims=['yball_relative'], label = genotype), aggregator='mean') for genotype in GroupData.Genotype.unique()])

Areas = hv.Overlay([hd.datashade(hv.Area(GroupData.query("Genotype == @genotype"), kdims=['time'], vdims=['ci_lower','ci_upper','Genotype'], label = genotype), aggregator='mean', cmap=my_cmap) for genotype in GroupData.Genotype.unique()])

hint_text = hv.Text(100, 5000, 'How to navigate: click on genotype labels in the legend to display/mute them. Refreshing can be slow, do not click multiple times. Downloading the plot locally makes it easier to navigate.', halign='right')

AvgCurves = (GenoCurves * Areas * hint_text).opts(width=1280, height=720, show_legend=True, xlabel="Time(s)",
        ylabel="Average relative distance pushed (px)",
        show_grid=True,
        fontscale=1.5,
        title="",)
hv.save(
    AvgCurves,
    "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/AverageCurvesTNTHV.html",
    #fmt="html",
)


In [ ]:
GenoCurves.keys

In [ ]:
bokeh.io.save(p, Savepath/"AverageBallpushingTNT.html")

In [ ]:
from bokeh.models import Model

if 'plot' in globals():
    for model in plot.select({'type': Model}):
        prev_doc = model.document
        model._document = None
        if prev_doc:
            prev_doc.remove_root(model)

In [ ]:
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.layouts import column

# Create a CheckboxGroup
checkbox_group = CheckboxGroup(labels=genotypes.tolist(), active=list(range(len(genotypes))))

# For each genotype, create a line plot with confidence intervals
for i, genotype in enumerate(genotypes):
    df_genotype = GroupData[GroupData['Genotype'] == genotype]
    
    # Downsample your data by taking every nth point
    
    n = 30
    df_genotype = df_genotype.iloc[::n]  # replace n with the desired sampling rate
    
    source = ColumnDataSource(df_genotype)
    
    # Draw the line for yball_relative
    line = p.line('time', 'yball_relative', source=source, line_width=2, color=Spectral11[i], legend_label=genotype)
    
    # Draw the upper and lower bounds for the confidence interval
    varea = p.varea(x='time', y1='ci_lower', y2='ci_upper', source=source, fill_color=Spectral11[i], fill_alpha=0.2)

    # Add a callback to the checkbox group to hide/show the line and varea
    checkbox_group.js_on_change("active", CustomJS(args=dict(line=line, varea=varea), code="""
        line.visible = cb_obj.active.includes(i);
        varea.visible = cb_obj.active.includes(i);
    """))

# Enable WebGL
#p.output_backend = "webgl"

# Show the results
#show(column(checkbox_group, p))
plot = column(checkbox_group, p)
bokeh.io.save(plot, Savepath/"AverageBallpushingTNT_check.html")


In [ ]:
CheckData = Dataset.loc[Dataset['Fly'] == 'Fly 1']

CheckData.head()

In [ ]:
CheckData['yball_smooth'].min()

## Implementing interaction metrics and plotting

In this part, we reuse the code from ConcatClips.py to get the interaction events from each video, and using these we can quantify metrics listed in 'Plotting ideas' in my notebook. These are :

* Number of events
* Event at which the ball is brought to the end
* Time at which the ball is brought to the end
* Amount of significant events
* Push vs pull 
* plot the interactions chronology
* time between interactions
* First meaningful interaction



Fixing the Event function :

In [ ]:
Dataset = Dataset.reset_index()

Dataset.head()

In [ ]:
TestData = Dataset.loc[Dataset['Fly'] == 'Fly 1']

In [ ]:
TestData.head()

In [ ]:
def test_find_interaction_events(df, Thresh, min_time):
    df.loc[:, "dist"] = df.loc[:, "yfly_smooth"] - df.loc[:, "yball_smooth"]
    df.loc[:, "close"] = df.loc[:,  "dist"] < Thresh
    df.loc[:, "block"] = (df.loc[:, "close"].shift(1) != df.loc[:, "close"]).cumsum()
    events = (
        df[df["close"]]
        .groupby("block")
        .agg(start=("index", "min"), end=("index", "max"))
    )
    interaction_events = [
        (start, end)
        for start, end in events[["start", "end"]].itertuples(index=False)
    ]
    interaction_events = [
        event for event in interaction_events if event[1] - event[0] >= min_time
    ]
    return interaction_events


In [ ]:
events = test_find_interaction_events(TestData, 80, 60)

events

In [ ]:
len(events)

In [ ]:
TestData2 = Dataset.loc[Dataset['Fly'] == 'Fly 2', :]

events2 = test_find_interaction_events(TestData2, 80, 60)

events2

In [ ]:
def test_extract_interaction_events(source,Thresh=80, min_time=60, as_df = False):
    if isinstance(source, Path):
        print(f"Path: {source}")
        flypath = next(source.glob("*tracked_fly*.analysis.h5"))
        ballpath = next(source.glob("*tracked*.analysis.h5"))
        df = get_coordinates(flypath=flypath, ballpath=ballpath)

    elif isinstance(source, pd.DataFrame):
        print(f"DataFrame: {source.shape}")
        df = source
        
    else:
        raise TypeError(
            "Invalid source format: source must be a pathlib Path, string or a pandas DataFrame"
        )

    # Create a new column 'Event' and initialize it with None
    df.loc[:, "Event"] = None

    if 'Fly' in df.columns:
        flies = df['Fly'].unique()
        for fly in flies:
            print(f"Processing {fly}...")

            # Compute interaction events for this fly only
            interaction_events_fly = test_find_interaction_events(df.loc[df['Fly'] == fly, :], Thresh, min_time)
            
            print (f'found {len(interaction_events_fly)} events')

            # Assign an event number to each event
            for i, (start_time, end_time) in enumerate(interaction_events_fly, start=1):
                df.loc[(df['Fly'] == fly) & (df.index >= start_time) & (df.index <= end_time), "Event"] = i
                
            NumbEvents = df.loc[df['Fly'] == fly, 'Event'].nunique()
            print(f"Number of events: {NumbEvents}")
    else:
        # Compute interaction events for all data
        interaction_events = test_find_interaction_events(df, Thresh, min_time)

        # Assign an event number to each event
        for i, (start_time, end_time) in enumerate(interaction_events, start=1):
            df.loc[(df.index >= start_time) & (df.index <= end_time), "Event"] = i

    if as_df:
        return df
    else:
        return interaction_events

In [ ]:
flies = Dataset['Fly'].unique()

flies = flies [:3]
# Generate a dataset with only the first 3 flies
Dataset_flies = Dataset[Dataset['Fly'].isin(flies)].reset_index()

Dataset_flies.head()

In [ ]:
test_extract_interaction_events(Dataset_flies, Thresh=80, min_time=60, as_df = True)

In [ ]:
len(events)

In [ ]:
# Reload the module
importlib.reload(Utilities.Ballpushing_utils)

# Now re-import your functions
from Utilities.Ballpushing_utils import *

In [ ]:
Dataset = extract_interaction_events(Dataset, as_df = True)
Dataset.head()

### Average number of events

In [ ]:
# For each Fly, compute how many unique values of Event that are not None there are

# Group the data by Fly and Event
GroupedData = Dataset.groupby(["Fly", "Genotype"]).nunique(['Event']).reset_index()

# Count the number of unique values of Event for each Fly
#Counts = GroupedData["Event"].nunique()
GroupedData.head()

In [ ]:
p_NumbEvents = iqplot.stripbox(data=GroupedData, q="Event", cats="Genotype", title="Number of events", frame_width= 750, frame_height= 500, tooltips=[("Event", "@{Event}"),("Genotype", "@{Genotype}")], spread="jitter")

show(p_NumbEvents)

In [ ]:
Savepath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/InteractionEvents/")


In [ ]:
# Holoviews version

h_NumbEvents_bp = hv.BoxWhisker(data=GroupedData, vdims="Event", kdims="Genotype", color = "Genotype").opts(box_fill_color=None, box_line_color="black", outlier_fill_color=None, outlier_line_color=None)
h_NumbEvents_sc = hv.Scatter(data=GroupedData, vdims="Event", kdims="Genotype", color = "Genotype").opts(jitter=0.3, color= "Genotype", alpha = 0.5, size = 6, cmap="Category10",  tools = ['hover'])

hvplot_NumbEvents = (h_NumbEvents_bp * h_NumbEvents_sc).opts(width=750, height=500, show_legend=False, xlabel="",
        ylabel="Number of Events",
        invert_axes=True,
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None)

hvplot_NumbEvents

In [ ]:
hv.save(
    hvplot_NumbEvents,
    Savepath/"NumberEvents.png",
)

In [ ]:
Sub = GroupedData.loc[GroupedData['Genotype'].isin(visible_genotypes), :]

# Holoviews version

h_NumbEvents_bp = hv.BoxWhisker(data=Sub, vdims="Event", kdims="Genotype", color = "Genotype").opts(box_fill_color=None, box_line_color="black", outlier_fill_color=None, outlier_line_color=None)
h_NumbEvents_sc = hv.Scatter(data=Sub, vdims="Event", kdims="Genotype", color = "Genotype").opts(jitter=0.3, color= "Genotype", alpha = 0.5, size = 6, cmap="Category10",  tools = ['hover'])

hvplot_NumbEvents_sub = (h_NumbEvents_bp * h_NumbEvents_sc).opts(width=750, height=500, show_legend=False, xlabel="",
        ylabel="Number of Events",
        invert_axes=True,
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None)

hvplot_NumbEvents_sub


hv.save(
    hvplot_NumbEvents_sub,
    Savepath/"NumberEvents_sub.png",
)


In [ ]:
bokeh.io.save(p_NumbEvents, filename=Savepath / "NumbEvents.png")

In [ ]:
p_NumbEvents.toolbar_location = None
bokeh.io.export_png(p_NumbEvents, filename=str(Savepath / "NumbEvents.png"))

In [ ]:
average_unique_events = Dataset.groupby(['Fly', 'Genotype'])['Event'].nunique().groupby('Genotype').mean()

#average_unique_events


### Event at which the ball is brough to the end

In [ ]:
end_threshold = 10

# Group by 'Fly' column and find the maximum 'yball_relative' for each group
max_positions = Dataset.groupby('Fly')['yball_relative'].max()

# Initialize an empty DataFrame to store the results
result = []

# For each Fly, find the first event where 'yball_relative' is equal to max_position
for fly, max_position in max_positions.items():
    print(f'{fly} has a max position of {max_position}')
    fly_data = Dataset[Dataset['Fly'] == fly]
    fly_data = fly_data.sort_values('Event')
    event = fly_data[fly_data['yball_relative'] >= max_position - end_threshold]['Event'].drop_duplicates().iloc[0]
    result.append({'Fly': fly, 'Event': event})

# Convert the result to a DataFrame
result_df = pd.DataFrame(result)

# Merge the result_df with the original Dataset
Dataset = pd.merge(Dataset, result_df, on=['Fly', 'Event'], how='left')

# Create the 'IsFinal' column, which is True if 'Event' is in result_df and False otherwise
Dataset['IsFinal'] = False
Dataset.loc[Dataset['Event'].isin(result_df['Event']), 'IsFinal'] = True

Dataset.head()


In [ ]:
Dataset.groupby('Fly')['yball_relative'].max()


In [ ]:
Dataset['max_position'] = Dataset.groupby('Fly')['yball_relative'].transform('max')

end_threshold = 10
Dataset['IsFinal'] = Dataset['yball_relative'] >= (Dataset['max_position'] - end_threshold)

Dataset.head()

In [ ]:
Dataset["Event"]

In [ ]:
# Drop rows where 'Event' is NaN
#Dataset = Dataset.dropna(subset=['Event'])

# Now extract the event number from the 'Event' column
#Dataset['EventNumber'] = Dataset['Event'].str.extract('(\d+)').astype(int)

# Filter rows where 'IsFinal' is True
final_events_trimmed = Dataset[Dataset['IsFinal'] == True]

#final_events_trimmed = final_events_trimmed.dropna(subset=['Event'])
final_events_trimmed.head()


In [ ]:
first_final = final_events_trimmed.groupby(['Fly'])['Event'].min()
first_final

In [ ]:
# Create a new column 'FinalEvent' in the original DataFrame
Dataset = Dataset.merge(first_final.rename('FinalEvent'), left_on='Fly', right_index=True, how='left')

Dataset.head()

In [ ]:
# Get unique values of Final event grouped by Fly
Dataset.groupby('Fly')['FinalEvent'].unique()

In [ ]:
# Get all values of Event for final_events grouped by Fly
final_events_trimmed.groupby('Fly')['Event'].unique()


In [ ]:

# For each Fly, find the minimum 'EventNumber' where 'IsFinal' is True
result = []
for fly in final_events['Fly'].unique():
    fly_data = final_events[final_events['Fly'] == fly]
    min_event_number = fly_data['Event'].min()
    result.append({'Fly': fly, 'MinEventNumber': min_event_number})

# Convert the result to a DataFrame
result_df = pd.DataFrame(result)


In [ ]:
# Create a new DataFrame with one row per fly
data_per_fly = Dataset.drop_duplicates(subset='Fly')

# Plot the data
p_finalevent = iqplot.stripbox(data=data_per_fly, q="FinalEvent", cats="Genotype", title="Event at which the fly reaches the end of the corridor", frame_width= 750, frame_height= 500, tooltips=[("FinalEvent", "@{FinalEvent}"),("Genotype", "@{Genotype}")], spread="jitter")

bokeh.io.show(p_finalevent)

#TODO: Figure out why this doesn't work properly


In [ ]:

# Save the plot as html
bokeh.io.save(
    p_finalevent,
    Savepath / "FinalEvent.html",
    #fmt="html",
)

In [ ]:
# Create a new DataFrame with rows where 'Event' equals 'FinalEvent'
final_event_rows = Dataset[Dataset['Event'] == Dataset['FinalEvent']]

# Drop duplicates based on 'Fly' to keep only the first occurrence of each fly
final_event_rows = final_event_rows.drop_duplicates(subset='Fly')


In [ ]:
p_final_event_time = iqplot.stripbox(data=final_event_rows, q="time", cats="Genotype", title="Time at which the fly reaches the end of the corridor", frame_width= 750, frame_height= 500, tooltips=[("time", "@{time}"),("Genotype", "@{Genotype}")], spread="jitter")

bokeh.io.show(p_final_event_time)


In [ ]:
# Holoviews version

h_TimeLastEvents_bp = hv.BoxWhisker(data=final_event_rows, vdims="time", kdims="Genotype", color = "Genotype").opts(box_fill_color=None, box_line_color="black", outlier_fill_color=None, outlier_line_color=None)
h_TimeLastEvents_sc = hv.Scatter(data=final_event_rows, vdims="time", kdims="Genotype", color = "Genotype").opts(jitter=0.3, color= "Genotype", alpha = 0.5, size = 6, cmap="Category10",  tools = ['hover'])

hvplot_TimeLastEvents = (h_TimeLastEvents_bp * h_TimeLastEvents_sc).opts(width=750, height=500, show_legend=False, xlabel="",
        ylabel="Time(s)",
        invert_axes=True,
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None)

hvplot_TimeLastEvents

Save dataset

In [ ]:
hv.save(
    hvplot_TimeLastEvents,
    Savepath/"TimeLastEvents.png",
)

In [ ]:
Sub_time = final_event_rows.loc[final_event_rows['Genotype'].isin(visible_genotypes), :]

h_TimeLastEvents_bp = hv.BoxWhisker(data=Sub_time, vdims="time", kdims="Genotype", color = "Genotype").opts(box_fill_color=None, box_line_color="black", outlier_fill_color=None, outlier_line_color=None)
h_TimeLastEvents_sc = hv.Scatter(data=Sub_time, vdims="time", kdims="Genotype", color = "Genotype").opts(jitter=0.3, color= "Genotype", alpha = 0.5, size = 6, cmap="Category10",  tools = ['hover'])

hvplot_TimeLastEvents_sub = (h_TimeLastEvents_bp * h_TimeLastEvents_sc).opts(width=750, height=500, show_legend=False, xlabel="",
        ylabel="Time(s)",
        invert_axes=True,
        show_grid=True,
        fontscale=1.5,
        title="",
        toolbar=None)

hvplot_TimeLastEvents_sub

In [ ]:

# Holoviews version
hv.save(
    hvplot_TimeLastEvents_sub,
    Savepath/"TimeLastEvents_sub.png",
)

In [ ]:
p_final_event_time.toolbar_location = None
bokeh.io.export_png(p_final_event_time, filename=str(Savepath / "FinalEventTime.png"))

In [ ]:
bokeh.io.save(
    p_final_event_time,
    Savepath / "FinalTime.html",
    #fmt="html",
)

In [ ]:
DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Datasets")

Dataset.to_feather(DataPath / "231002_DatasetTNT_InteractionEvents.feather")

In [ ]:
def check_yball_variation(event_df, threshold=10):
    yball_segment = event_df["yball_smooth"]
    variation = yball_segment.max() - yball_segment.min()
    return variation > threshold

# Apply the function and reset the index
significant_events = Dataset.groupby(["Fly", "Event"]).apply(check_yball_variation).reset_index()

# Rename the 0 column to 'SignificantEvent'
significant_events.rename(columns={0: 'SignificantEvent'}, inplace=True)

# Merge the significant_events DataFrame with the original Dataset
Dataset = pd.merge(Dataset, significant_events, on=['Fly', 'Event'], how='left')

Dataset.head()


In [ ]:
# Count the number of unique events that are significant for each 'Fly' and 'Genotype'
unique_significant_events = Dataset[Dataset['SignificantEvent'] == True].groupby(['Fly', 'Genotype'])['Event'].nunique()

#print(unique_significant_events)


In [ ]:
p_unique_significant_events = iqplot.stripbox(data=unique_significant_events.reset_index(), q="Event", cats="Genotype", title="Number of significant events", frame_width= 750, frame_height= 500,  tooltips=[("Event", "@{Event}"),("Genotype", "@{Genotype}")], spread="jitter")

bokeh.io.show(p_unique_significant_events)

In [ ]:
bokeh.io.save(
    p_unique_significant_events,
    Savepath / "NumbSignificantEvents.html",
    #fmt="html",
)

In [ ]:
# Filter the DataFrame to include only the significant events
significant_events = Dataset[Dataset['SignificantEvent'] == True]

# For each 'Fly', find the first significant event and get its time
first_significant_event_times = significant_events.groupby('Fly')['time'].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]

first_significant_events.head()


In [ ]:
# For each 'Fly' and 'Genotype', find the first significant event and get its time
first_significant_event_times = significant_events.groupby(['Fly', 'Genotype'])['time'].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]

# Plot the data
p_first_significant_event_times = iqplot.stripbox(data=first_significant_events.reset_index(), q="time", cats="Genotype", title="Time of first significant event", frame_width= 750, frame_height= 500,   tooltips=[("time", "@{time}"),("Genotype", "@{Genotype}")], spread="jitter")
bokeh.io.show(p_first_significant_event_times)


In [ ]:
bokeh.io.save(
    p_first_significant_event_times,
    Savepath / "FirstSignificantEvent_Time.html",
    #fmt="html",
)

In [ ]:
# Filter the DataFrame to include only the significant events
significant_events = Dataset[Dataset['SignificantEvent'] == True]

# For each 'Fly', find the first significant event and get its time
first_significant_event_times = significant_events.groupby('Fly')['time'].idxmin()

# Use these indices to get the corresponding rows from the original DataFrame
first_significant_events = Dataset.loc[first_significant_event_times]

#first_significant_events["EventNumber"] = first_significant_events.loc[:,"Event"].str.extract('(\d+)').astype(int)

first_significant_events.head()



In [ ]:
# Plot the data
p_first_significant_event_numbers = iqplot.stripbox(data=first_significant_events.reset_index(), q="Event", cats="Genotype", title="Number of first significant event", frame_width= 750, frame_height= 500, tooltips=[("Event", "@{Event}"),("Genotype", "@{Genotype}")], spread="jitter")
bokeh.io.show(p_first_significant_event_numbers)

In [ ]:
bokeh.io.save(
    p_first_significant_event_numbers,
    Savepath / "FirstSignificantEvent_Number.html",
    #fmt="html",
)

# Pauses

In [ ]:
import numpy as np
Dataset['Event'] = Dataset['Event'].replace('None', np.nan)


In [ ]:
Dataset['IsEvent'] = Dataset['Event'].notna()


In [ ]:
Dataset['TimeSinceLastEvent'] = Dataset.groupby('Fly').apply(lambda x: x['time'] - x.loc[x['IsEvent'], 'time'].shift()).reset_index(level=0, drop=True)


In [ ]:
## For each 'Fly', calculate the time difference between consecutive events
Dataset['TimeSinceLastEvent'] = Dataset[Dataset['IsEvent']].groupby('Fly')['time'].diff()

# Calculate the average time between events for each 'Fly' and 'Genotype'
average_time_between_events = Dataset.groupby(['Fly', 'Genotype'])['TimeSinceLastEvent'].mean().reset_index()

# Plot the data
p_average_time_between_events = iqplot.stripbox(data=average_time_between_events, q="TimeSinceLastEvent", cats="Genotype", title="Average time between events", frame_width= 750, frame_height= 500, tooltips=[("TimeSinceLastEvent", "@{TimeSinceLastEvent}"),("Genotype", "@{Genotype}")], spread="jitter")
bokeh.io.show(p_average_time_between_events)


In [ ]:
bokeh.io.save(
    p_average_time_between_events,
    Savepath / "AverageTimeBetweenEvents.html",
    #fmt="html",
)

In [ ]:
Dataset['IsNonEvent'] = Dataset['Event'].isna()


In [ ]:
# Calculate the time difference between consecutive rows
Dataset['TimeDifference'] = Dataset.groupby('Fly')['time'].diff()

# For non-event rows, sum these time differences
Dataset['NonEventTime'] = Dataset.loc[Dataset['IsNonEvent'], 'TimeDifference']

# Calculate the cumulative time spent in non-events for each 'Fly' and 'Genotype'
cumulative_time_in_non_events = Dataset.groupby(['Fly', 'Genotype'])['NonEventTime'].sum().reset_index()

# Plot the data
p_cumulative_time_in_non_events = iqplot.stripbox(data=cumulative_time_in_non_events, q="NonEventTime", cats="Genotype", title="Cumulative time spent not interacting", frame_width= 750, frame_height= 500, tooltips=[("NonEventTime", "@{NonEventTime}"),("Genotype", "@{Genotype}")], spread="jitter")
bokeh.io.show(p_cumulative_time_in_non_events)


In [ ]:
bokeh.io.save(
    p_cumulative_time_in_non_events,
    Savepath / "CumulativeTimeNonEvents.html",
    #fmt="html",
)

In [ ]:
# For each 'Fly', 'Event', and 'Genotype', get the timestamp of the first frame
first_frame_timestamps = Dataset.groupby(['Fly', 'Event', 'Genotype'])['time'].first().reset_index()

# Plot the data
p_first_frame_timestamps = iqplot.histogram(data=first_frame_timestamps, q="time", cats="Genotype", title="Timestamp of first frame of each event", frame_width= 750, frame_height= 2000, )
bokeh.io.show(p_first_frame_timestamps)


In [ ]:
bokeh.io.save(
    p_first_frame_timestamps,
    Savepath / "FirstFrameTimestamps.html",
    #fmt="html",
)

### Puls

In [ ]:
# Calculate the difference in 'yball_relative' between the start and end of each event
yball_relative_diff = Dataset.groupby(['Fly', 'Event'])['yball_relative'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index(name='yball_relative_diff')

# Merge yball_relative_diff with Dataset
Dataset = pd.merge(Dataset, yball_relative_diff, on=['Fly', 'Event'], how='left')


In [ ]:

Dataset.head()


In [ ]:

# Identify whether each event is a 'Push', 'Pull', or 'None'
Dataset['Event_Type'] = np.where(Dataset['yball_relative_diff'] > 10, 'Push', np.where(Dataset['yball_relative_diff'] < -10, 'Pull', 'None'))

# Count the number of each event type for each 'Fly' and 'Genotype'
event_type_counts = Dataset.groupby(['Fly', 'Genotype', 'Event_Type']).size().reset_index(name='Count')

# Count the number of unique events with event type for each fly and genotype
unique_event_type_counts = event_type_counts.groupby(['Fly', 'Genotype'])['Event_Type'].nunique().reset_index(name='Count')

# Filter the DataFrame to include only the 'Pull' events
pull_events = Dataset[Dataset['Event_Type'] == 'Pull']

# Count the number of 'Pull' events for each 'Fly' and 'Genotype'
pull_event_counts = pull_events.groupby(['Fly', 'Genotype']).size().reset_index(name='Count')

# Count the number of unique 'Pull' events for each 'Fly' and 'Genotype'
unique_pull_event_counts = pull_events.groupby(['Fly', 'Genotype'])['Event'].nunique().reset_index(name='Count')

# Plot the data


p_pull_event_counts = iqplot.stripbox(data=unique_pull_event_counts, q="Count", cats="Genotype", title="Number of 'Pull' events", frame_width= 750, frame_height= 500, tooltips=[("Count", "@{Count}"),("Genotype", "@{Genotype}")], spread="jitter")
bokeh.io.show(p_pull_event_counts)



In [ ]:
bokeh.io.save(
    p_pull_event_counts,
    Savepath / "PullEventCounts.html",
    #fmt="html",
)

In [ ]:
#TODO : Pull and push within event

#TODO : Make a dashboard with all the plots

# Making a dashboard

## Chapters


*Interactions chronology



In [ ]:
# Make a list of the plots to include in the dashboard

Plots = [p_NumbEvents, p_finalevent, p_final_event_time, p_unique_significant_events, p_first_significant_event_times, p_first_significant_event_numbers, p_average_time_between_events, p_cumulative_time_in_non_events, p_first_frame_timestamps, p_pull_event_counts]


In [ ]:
from bokeh.models import Model

for p in Plots:
    for model in p.select({'type': Model}):
        prev_doc = model.document
        model._document = None
        if prev_doc:
            prev_doc.remove_root(model)


In [ ]:

# Create a gridplot of the plots

grid = bokeh.layouts.gridplot([[p_first_frame_timestamps, p_NumbEvents],
                               [None, p_cumulative_time_in_non_events],
                               [None, p_unique_significant_events],
                               [p_finalevent, p_final_event_time],
                               [p_first_significant_event_numbers, p_first_significant_event_times],
                               [p_average_time_between_events, p_pull_event_counts]])

bokeh.io.save(grid,
              Savepath / "Dashboard.html")

In [ ]:
from bokeh.layouts import column, row

# Create a column layout for the large plot
column1 = column(p_first_frame_timestamps)

# Create a column layout for the smaller plots
column2 = column(p_NumbEvents, p_cumulative_time_in_non_events, p_unique_significant_events)

# Put the two columns into a row layout
row1 = row(column1, column2)

# Create a gridplot for the remaining plots
grid = bokeh.layouts.gridplot([[row1], [p_finalevent, p_final_event_time],
                               [p_first_significant_event_numbers, p_first_significant_event_times],
                               [p_average_time_between_events, p_pull_event_counts]])

bokeh.io.save(grid, Savepath / "Dashboard.html")
